In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.optimize import minimize

# Disable interactive mode
plt.ioff()

# Define paths
ice_folder_path = r"C:\Users\yliu117\Desktop\EV\ACC_data"
ev_folder_path = r"C:\Users\yliu117\Desktop\EV\EV-ACC data\combined"
new_dataset_path = r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025"
output_base_path = r"C:\Users\yliu117\Desktop\EV_vs_ICE"

# Initial OVRV parameters for initial guess
initial_params = {
    "ICE": {
        "A": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 6.0, "tau": 2.0}},
        "B": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 7.0, "tau": 2.0}},
        "C": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 7.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 2.0}},
        "D": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 2.0}},
        "E": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 4.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 2.0}},
        "F": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 7.0, "tau": 2.0}},
        "G": {"Min": {"k1": 0.1, "k2": 0.5, "eta": 8.0, "tau": 1.5},
              "Max": {"k1": 0.1, "k2": 0.5, "eta": 4.0, "tau": 2.0}}
    },
    "EV": {
        "Short": {"k1": 0.2, "k2": 0.6, "eta": 2.0, "tau": 1.5},
        "Medium": {"k1": 0.2, "k2": 0.6, "eta": 2.0, "tau": 1.6},
        "Long": {"k1": 0.2, "k2": 0.6, "eta": 2.5, "tau": 1.7},
        "Xlong": {"k1": 0.2, "k2": 0.6, "eta": 5.0, "tau": 2.0}
    },
    "Polestar_Short": {"k1": 0.2, "k2": 0.6, "eta": 2.0, "tau": 1.5},
    "Tesla_Short": {"k1": 0.2, "k2": 0.6, "eta": 2.0, "tau": 1.5}
}

# Parameter bounds
bounds = {
    'k1': (0.01, 1.0),
    'k2': (0.1, 2.0),
    'eta': (1.0, 10.0),
    'tau': (0.5, 3.0)
}

# Plot parameters
plot_params = {
    'xlabel_fontsize': 10,
    'ylabel_fontsize': 10,
    'legend_fontsize': 7,
    'tick_labelsize': 8,
    'grid_linestyle': '--',
    'grid_alpha': 0.7,
    'title_fontsize': 7
}

# Color map for conditions
ice_color_map = {
    "dip_min": "blue",
    "osc_min": "purple",
    "stepH_min": "cyan",
    "stepL_min": "brown",
    "dip_max": "red",
    "osc_max": "green",
    "stepH_max": "orange",
    "stepL_max": "magenta"
}
ev_color_map = {
    "35": sns.husl_palette(3)[0],
    "45": sns.husl_palette(3)[1],
    "55": sns.husl_palette(3)[2]
}
new_dataset_color_map = {
    "55_25": sns.husl_palette(6)[0],
    "55_35": sns.husl_palette(6)[1],
    "55_45": sns.husl_palette(6)[2],
    "45_25": sns.husl_palette(6)[3],
    "45_35": sns.husl_palette(6)[4],
    "55_35_5_desired": sns.husl_palette(6)[5]
}

# OVRV model function
def ovrv_acceleration(v, delta_v, s, k1, k2, eta, tau):
    acc = k1 * (s - eta - tau * v) + k2 * delta_v
    return acc

# Simulate OVRV trajectory
def simulate_ovrv(lead_speed, lead_time, params, initial_v, initial_s, dt):
    k1 = params['k1']
    k2 = params['k2']
    eta = params['eta']
    tau = params['tau']
   
    v = [initial_v]
    s = [initial_s]
    t = [lead_time[0]]
   
    for i in range(1, len(lead_time)):
        delta_v = lead_speed[i] - v[-1]
        acc = ovrv_acceleration(v[-1], delta_v, s[-1], k1, k2, eta, tau)
        v_new = v[-1] + acc * dt
        v_new = max(v_new, 0) # Prevent negative speed
        s_new = s[-1] + (lead_speed[i] - v_new) * dt
        s_new = max(s_new, eta) # Prevent negative or too small spacing
        v.append(v_new)
        s.append(s_new)
        t.append(t[-1] + dt)
   
    return np.array(t), np.array(v), np.array(s)

# Calculate RMSE
def calculate_rmse(actual, predicted):
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted)
    if np.sum(valid_mask) == 0:
        return np.nan
    return np.sqrt(np.mean((actual[valid_mask] - predicted[valid_mask]) ** 2))

# Objective function for optimization (sum of spacing RMSE across conditions)
def objective_function(params, datasets, dt):
    k1, k2, eta, tau = params
    ovrv_params = {'k1': k1, 'k2': k2, 'eta': eta, 'tau': tau}
    total_rmse = 0.0
   
    for time, lead_speed, follow_speed, spacing in datasets:
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else eta
        ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, ovrv_params, initial_v, initial_s, dt)
        spacing_rmse = calculate_rmse(spacing, ovrv_spacing)
        if not np.isnan(spacing_rmse):
            total_rmse += spacing_rmse
   
    return total_rmse

# Create output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = os.path.join(output_base_path, f"OVRV_Calibration_{timestamp}")
os.makedirs(output_folder, exist_ok=True)

# Vehicle and condition definitions
ice_vehicles = ["A", "B", "C", "D", "E", "F", "G"]
ice_settings = ["Min", "Max"]
ice_conditions = {
    "Min": ["dip_min", "osc_min", "stepH_min", "stepL_min"],
    "Max": ["dip_max", "osc_max", "stepH_max", "stepL_max"]
}
ev_vehicles = ["Short", "Medium", "Long", "Xlong"]
ev_conditions = ["35", "45", "55"]
new_datasets = {
    "Polestar_Short": [
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\25\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\35\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\3.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\4.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Polestar\Short\0_desired\55\45\5.csv"
    ],
    "Tesla_Short": [
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\45\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\25\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\25\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\35\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\45\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\0_desired\55\45\2.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\5_desired\55\35\1.csv",
        r"C:\Users\yliu117\Desktop\EV\EV-main\OneDrive_1_10-1-2025\Tesla\Short\5_desired\55\35\2.csv"
    ]
}
new_conditions = {
    "Polestar_Short": ["55_25", "55_25", "55_25", "55_35", "55_35", "55_35", "55_45", "55_45", "55_45", "55_45", "55_45"],
    "Tesla_Short": ["45_25", "45_25", "45_35", "55_25", "55_25", "55_35", "55_35", "55_45", "55_45", "55_35_5_desired", "55_35_5_desired"]
}

# Store best parameters and RMSEs
best_params = {"ICE": {}, "EV": {}, "New_Datasets": {}}
rmse_results = {"ICE": {}, "EV": {}, "New_Datasets": {}}

# Optimize parameters for ICE vehicles
for vehicle in ice_vehicles:
    for setting in ice_settings:
        # Load datasets for all conditions
        datasets = []
        for condition in ice_conditions[setting]:
            csv_file = f"Veh{vehicle}_{condition}.csv"
            try:
                df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
                time = df.iloc[:, 0].values
                lead_speed = df.iloc[:, 2].values # Lead Speed (m/s)
                follow_speed = df.iloc[:, 1].values # Follow Speed (m/s)
                spacing = df.iloc[:, 3].values # Spacing (m)
                datasets.append((time, lead_speed, follow_speed, spacing))
            except Exception as e:
                print(f"Error reading {csv_file}: {e}")
                continue
       
        # Initial guess from initial_params
        initial_guess = [
            initial_params["ICE"][vehicle][setting]['k1'],
            initial_params["ICE"][vehicle][setting]['k2'],
            initial_params["ICE"][vehicle][setting]['eta'],
            initial_params["ICE"][vehicle][setting]['tau']
        ]
       
        # Bounds for optimization
        param_bounds = [
            bounds['k1'],
            bounds['k2'],
            bounds['eta'],
            bounds['tau']
        ]
       
        # Optimize parameters
        result = minimize(
            objective_function,
            initial_guess,
            args=(datasets, 0.1),
            method='SLSQP',
            bounds=param_bounds,
            options={'disp': True, 'maxiter': 1000}
        )
       
        # Store best parameters
        if vehicle not in best_params["ICE"]:
            best_params["ICE"][vehicle] = {}
        best_params["ICE"][vehicle][setting] = {
            'k1': result.x[0],
            'k2': result.x[1],
            'eta': result.x[2],
            'tau': result.x[3]
        }
       
        # Calculate RMSE for each condition
        rmse_results["ICE"][(vehicle, setting)] = {}
        for condition, (time, lead_speed, follow_speed, spacing) in zip(ice_conditions[setting], datasets):
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
            initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["ICE"][vehicle][setting]['eta']
            ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["ICE"][vehicle][setting], initial_v, initial_s, 0.1)
            speed_rmse = calculate_rmse(follow_speed, ovrv_speed)
            spacing_rmse = calculate_rmse(spacing, ovrv_spacing)
            rmse_results["ICE"][(vehicle, setting)][condition] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Optimize parameters for EV vehicles
for vehicle in ev_vehicles:
    # Load datasets for all conditions
    datasets = []
    for condition in ev_conditions:
        csv_file = f"{vehicle}_{condition}.csv"
        try:
            df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778 # Convert km/h to m/s
            follow_speed = df['Smooth Speed Follower'].values * 0.27778 # Convert km/h to m/s
            spacing = df['Spacing'].values # Spacing in meters
            datasets.append((time, lead_speed, follow_speed, spacing))
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
            continue
   
    # Initial guess from initial_params
    initial_guess = [
        initial_params["EV"][vehicle]['k1'],
        initial_params["EV"][vehicle]['k2'],
        initial_params["EV"][vehicle]['eta'],
        initial_params["EV"][vehicle]['tau']
    ]
   
    # Bounds for optimization
    param_bounds = [
        bounds['k1'],
        bounds['k2'],
        bounds['eta'],
        bounds['tau']
    ]
   
    # Optimize parameters
    result = minimize(
        objective_function,
        initial_guess,
        args=(datasets, 0.02),
        method='SLSQP',
        bounds=param_bounds,
        options={'disp': True, 'maxiter': 1000}
    )
   
    # Store best parameters
    best_params["EV"][vehicle] = {
        'k1': result.x[0],
        'k2': result.x[1],
        'eta': result.x[2],
        'tau': result.x[3]
    }
   
    # Calculate RMSE for each condition
    rmse_results["EV"][vehicle] = {}
    for condition, (time, lead_speed, follow_speed, spacing) in zip(ev_conditions, datasets):
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["EV"][vehicle]['eta']
        ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["EV"][vehicle], initial_v, initial_s, 0.02)
        speed_rmse = calculate_rmse(follow_speed, ovrv_speed)
        spacing_rmse = calculate_rmse(spacing, ovrv_spacing)
        rmse_results["EV"][vehicle][condition] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Optimize parameters for new datasets (Polestar and Tesla)
for dataset_name in new_datasets:
    # Load datasets for all files
    datasets = []
    condition_labels = []
    for csv_file, condition in zip(new_datasets[dataset_name], new_conditions[dataset_name]):
        try:
            df = pd.read_csv(csv_file)
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778 # Convert km/h to m/s
            follow_speed = df['Smooth Speed Follower'].values * 0.27778 # Convert km/h to m/s
            spacing = df['Spacing'].values # Spacing in meters
            datasets.append((time, lead_speed, follow_speed, spacing))
            condition_labels.append(condition)
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
            continue
   
    # Initial guess from initial_params
    initial_guess = [
        initial_params[dataset_name]['k1'],
        initial_params[dataset_name]['k2'],
        initial_params[dataset_name]['eta'],
        initial_params[dataset_name]['tau']
    ]
   
    # Bounds for optimization
    param_bounds = [
        bounds['k1'],
        bounds['k2'],
        bounds['eta'],
        bounds['tau']
    ]
   
    # Optimize parameters
    result = minimize(
        objective_function,
        initial_guess,
        args=(datasets, 0.02),
        method='SLSQP',
        bounds=param_bounds,
        options={'disp': True, 'maxiter': 1000}
    )
   
    # Store best parameters
    best_params["New_Datasets"][dataset_name] = {
        'k1': result.x[0],
        'k2': result.x[1],
        'eta': result.x[2],
        'tau': result.x[3]
    }
   
    # Calculate RMSE for each condition
    rmse_results["New_Datasets"][dataset_name] = {}
    for condition, (time, lead_speed, follow_speed, spacing) in zip(condition_labels, datasets):
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["New_Datasets"][dataset_name]['eta']
        ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["New_Datasets"][dataset_name], initial_v, initial_s, 0.02)
        speed_rmse = calculate_rmse(follow_speed, ovrv_speed)
        spacing_rmse = calculate_rmse(spacing, ovrv_spacing)
        rmse_results["New_Datasets"][dataset_name][condition] = {'speed_rmse': speed_rmse, 'spacing_rmse': spacing_rmse}

# Print best parameters and RMSEs
print("\nBest OVRV Parameters and RMSEs for ICE Vehicles:")
for vehicle in ice_vehicles:
    for setting in ice_settings:
        print(f"\nVehicle: {vehicle}, Setting: {setting}")
        print(f"k1: {best_params['ICE'][vehicle][setting]['k1']:.4f} 1/s²")
        print(f"k2: {best_params['ICE'][vehicle][setting]['k2']:.4f} 1/s")
        print(f"eta: {best_params['ICE'][vehicle][setting]['eta']:.2f} m")
        print(f"tau: {best_params['ICE'][vehicle][setting]['tau']:.2f} s")
        print("RMSEs:")
        for condition in ice_conditions[setting]:
            print(f" Condition {condition}:")
            print(f" Speed RMSE: {rmse_results['ICE'][(vehicle, setting)][condition]['speed_rmse']:.2f} m/s")
            print(f" Spacing RMSE: {rmse_results['ICE'][(vehicle, setting)][condition]['spacing_rmse']:.2f} m")

print("\nBest OVRV Parameters and RMSEs for EV Vehicles:")
for vehicle in ev_vehicles:
    print(f"\nACC Setting: {vehicle}")
    print(f"k1: {best_params['EV'][vehicle]['k1']:.4f} 1/s²")
    print(f"k2: {best_params['EV'][vehicle]['k2']:.4f} 1/s")
    print(f"eta: {best_params['EV'][vehicle]['eta']:.2f} m")
    print(f"tau: {best_params['EV'][vehicle]['tau']:.2f} s")
    print("RMSEs:")
    for condition in ev_conditions:
        print(f" Condition {condition} km/h:")
        print(f" Speed RMSE: {rmse_results['EV'][vehicle][condition]['speed_rmse']:.2f} m/s")
        print(f" Spacing RMSE: {rmse_results['EV'][vehicle][condition]['spacing_rmse']:.2f} m")

print("\nBest OVRV Parameters and RMSEs for New Datasets:")
for dataset_name in new_datasets:
    print(f"\nDataset: {dataset_name}")
    print(f"k1: {best_params['New_Datasets'][dataset_name]['k1']:.4f} 1/s²")
    print(f"k2: {best_params['New_Datasets'][dataset_name]['k2']:.4f} 1/s")
    print(f"eta: {best_params['New_Datasets'][dataset_name]['eta']:.2f} m")
    print(f"tau: {best_params['New_Datasets'][dataset_name]['tau']:.2f} s")
    print("RMSEs:")
    for condition in set(new_conditions[dataset_name]):  # Use set to avoid duplicate prints
        files_with_condition = [f for f, c in zip(new_datasets[dataset_name], new_conditions[dataset_name]) if c == condition]
        for file_path in files_with_condition:
            file_name = os.path.basename(file_path)
            print(f" File {file_name} (Condition {condition}):")
            print(f" Speed RMSE: {rmse_results['New_Datasets'][dataset_name][condition]['speed_rmse']:.2f} m/s")
            print(f" Spacing RMSE: {rmse_results['New_Datasets'][dataset_name][condition]['spacing_rmse']:.2f} m")

# Plot ICE trajectories
fig_ice, axes_ice = plt.subplots(7, 16, figsize=(56, 36), sharex='col')
axes_ice = axes_ice.flatten()

# Calculate max time for ICE conditions
max_times_ice = {condition: 0 for condition in ice_conditions["Min"] + ice_conditions["Max"]}
for vehicle in ice_vehicles:
    for setting in ice_settings:
        for condition in ice_conditions[setting]:
            csv_file = f"Veh{vehicle}_{condition}.csv"
            try:
                df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
                time = df.iloc[:, 0].values
                valid_time_mask = ~np.isnan(time)
                time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
                max_times_ice[condition] = max(max_times_ice[condition], np.max(time_shifted[~np.isnan(time_shifted)]))
            except Exception as e:
                print(f"Error reading {csv_file} for max time: {e}")

# Plot for each ICE vehicle and condition
for idx, (vehicle, condition) in enumerate([(v, c) for v in ice_vehicles for c in ice_conditions["Min"] + ice_conditions["Max"]]):
    setting = "Min" if condition.endswith("_min") else "Max"
    csv_file = f"Veh{vehicle}_{condition}.csv"
    try:
        df = pd.read_csv(os.path.join(ice_folder_path, csv_file))
        time = df.iloc[:, 0].values
        lead_speed = df.iloc[:, 2].values
        follow_speed = df.iloc[:, 1].values
        spacing = df.iloc[:, 3].values
       
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["ICE"][vehicle][setting]['eta']
        ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["ICE"][vehicle][setting], initial_v, initial_s, 0.1)
       
        speed_rmse = rmse_results["ICE"][(vehicle, setting)][condition]['speed_rmse']
        spacing_rmse = rmse_results["ICE"][(vehicle, setting)][condition]['spacing_rmse']
        max_time = np.ceil(max(max_times_ice[condition], np.max(ovrv_time)))
       
        # Plot speed
        ax_speed = axes_ice[idx * 2]
        ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=ice_color_map[condition], linestyle='-', linewidth=1)
        ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=ice_color_map[condition], linestyle='--', linewidth=1)
        ax_speed.plot(ovrv_time, ovrv_speed, label="OVRV Follow Speed", color='black', linestyle=':', linewidth=1)
        ax_speed.set_title(
            f"Veh{vehicle}_{condition} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_speed.legend(
            ["Actual Follow Speed", "Lead Speed", "OVRV Follow Speed"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
        )
        ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
        ax_speed.set_xlim(0, max_time)
        ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 >= 96:
            ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
       
        # Plot spacing
        ax_spacing = axes_ice[idx * 2 + 1]
        ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=ice_color_map[condition], linestyle='-', linewidth=1)
        ax_spacing.plot(ovrv_time, ovrv_spacing, label="OVRV Spacing", color='black', linestyle=':', linewidth=1)
        ax_spacing.set_title(
            f"Veh{vehicle}_{condition} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_spacing.legend(
            ["Actual Spacing", "OVRV Spacing"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
        )
        ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
        ax_spacing.set_xlim(0, max_time)
        ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 + 1 >= 96:
            ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")

# Adjust ICE layout
fig_ice.suptitle("ICE Speed and Spacing Trajectories with Optimized OVRV Parameters", fontsize=14)
fig_ice.tight_layout(rect=[0, 0, 1, 0.95])
fig_ice.savefig(os.path.join(output_folder, "ice_trajectories_optimized.png"), dpi=300, bbox_inches='tight')
plt.close(fig_ice)

# Plot EV trajectories
fig_ev, axes_ev = plt.subplots(4, 6, figsize=(24, 16), sharex='col')
axes_ev = axes_ev.flatten()

# Calculate max time for EV conditions
max_times_ev = {condition: 0 for condition in ev_conditions}
for vehicle in ev_vehicles:
    for condition in ev_conditions:
        csv_file = f"{vehicle}_{condition}.csv"
        try:
            df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
            time = df['Time'].values
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            max_times_ev[condition] = max(max_times_ev[condition], np.max(time_shifted[~np.isnan(time_shifted)]))
        except Exception as e:
            print(f"Error reading {csv_file} for max time: {e}")

# Plot for each EV vehicle and condition
for idx, (vehicle, condition) in enumerate([(v, c) for v in ev_vehicles for c in ev_conditions]):
    csv_file = f"{vehicle}_{condition}.csv"
    try:
        df = pd.read_csv(os.path.join(ev_folder_path, csv_file))
        time = df['Time'].values
        lead_speed = df['Smooth Speed Leader'].values * 0.27778
        follow_speed = df['Smooth Speed Follower'].values * 0.27778
        spacing = df['Spacing'].values
       
        valid_time_mask = ~np.isnan(time)
        time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
        initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
        initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["EV"][vehicle]['eta']
        ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["EV"][vehicle], initial_v, initial_s, 0.02)
       
        speed_rmse = rmse_results["EV"][vehicle][condition]['speed_rmse']
        spacing_rmse = rmse_results["EV"][vehicle][condition]['spacing_rmse']
        max_time = np.ceil(max(max_times_ev[condition], np.max(ovrv_time)))
       
        # Plot speed
        ax_speed = axes_ev[idx * 2]
        ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=ev_color_map[condition], linestyle='-', linewidth=1)
        ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=ev_color_map[condition], linestyle='--', linewidth=1)
        ax_speed.plot(ovrv_time, ovrv_speed, label="OVRV Follow Speed", color='black', linestyle=':', linewidth=1)
        ax_speed.set_title(
            f"{vehicle}_{condition} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_speed.legend(
            ["Actual Follow Speed", "Lead Speed", "OVRV Follow Speed"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
        )
        ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
        ax_speed.set_xlim(0, max_time)
        ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 >= 16:
            ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
       
        # Plot spacing
        ax_spacing = axes_ev[idx * 2 + 1]
        ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=ev_color_map[condition], linestyle='-', linewidth=1)
        ax_spacing.plot(ovrv_time, ovrv_spacing, label="OVRV Spacing", color='black', linestyle=':', linewidth=1)
        ax_spacing.set_title(
            f"{vehicle}_{condition} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
            fontsize=plot_params['title_fontsize'], wrap=True
        )
        ax_spacing.legend(
            ["Actual Spacing", "OVRV Spacing"],
            fontsize=plot_params['legend_fontsize'], loc='upper left',
            bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
        )
        ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
        ax_spacing.set_xlim(0, max_time)
        ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
        ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
        if idx * 2 + 1 >= 16:
            ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")

# Adjust EV layout
fig_ev.suptitle("EV Speed and Spacing Trajectories with Optimized OVRV Parameters", fontsize=14)
fig_ev.tight_layout(rect=[0, 0, 1, 0.95])
fig_ev.savefig(os.path.join(output_folder, "ev_trajectories_optimized.png"), dpi=300, bbox_inches='tight')
plt.close(fig_ev)

# Plot new datasets trajectories
fig_new, axes_new = plt.subplots(11, 4, figsize=(16, 44), sharex='col')  # 11 rows for 11 files, 4 columns (2 per dataset)
axes_new = axes_new.flatten()

# Calculate max time for new dataset conditions
max_times_new = {condition: 0 for dataset_name in new_conditions for condition in set(new_conditions[dataset_name])}
for dataset_name in new_datasets:
    for csv_file, condition in zip(new_datasets[dataset_name], new_conditions[dataset_name]):
        try:
            df = pd.read_csv(csv_file)
            time = df['Time'].values
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            max_times_new[condition] = max(max_times_new[condition], np.max(time_shifted[~np.isnan(time_shifted)]))
        except Exception as e:
            print(f"Error reading {csv_file} for max time: {e}")

# Plot for each new dataset and condition
plot_idx = 0
for dataset_name in new_datasets:
    for csv_file, condition in zip(new_datasets[dataset_name], new_conditions[dataset_name]):
        try:
            df = pd.read_csv(csv_file)
            time = df['Time'].values
            lead_speed = df['Smooth Speed Leader'].values * 0.27778
            follow_speed = df['Smooth Speed Follower'].values * 0.27778
            spacing = df['Spacing'].values
       
            valid_time_mask = ~np.isnan(time)
            time_shifted = time - time[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else time
            initial_v = follow_speed[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else 0
            initial_s = spacing[valid_time_mask][0] if np.sum(valid_time_mask) > 0 else best_params["New_Datasets"][dataset_name]['eta']
            ovrv_time, ovrv_speed, ovrv_spacing = simulate_ovrv(lead_speed, time_shifted, best_params["New_Datasets"][dataset_name], initial_v, initial_s, 0.02)
       
            speed_rmse = rmse_results["New_Datasets"][dataset_name][condition]['speed_rmse']
            spacing_rmse = rmse_results["New_Datasets"][dataset_name][condition]['spacing_rmse']
            max_time = np.ceil(max(max_times_new[condition], np.max(ovrv_time)))
       
            # Plot speed
            ax_speed = axes_new[plot_idx * 2]
            ax_speed.plot(time_shifted, follow_speed, label="Actual Follow Speed", color=new_dataset_color_map[condition], linestyle='-', linewidth=1)
            ax_speed.plot(time_shifted, lead_speed, label="Lead Speed", color=new_dataset_color_map[condition], linestyle='--', linewidth=1)
            ax_speed.plot(ovrv_time, ovrv_speed, label="OVRV Follow Speed", color='black', linestyle=':', linewidth=1)
            ax_speed.set_title(
                f"{dataset_name}_{condition}_{os.path.basename(csv_file)} Speed\nSpeed RMSE: {speed_rmse:.2f} m/s",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_speed.legend(
                ["Actual Follow Speed", "Lead Speed", "OVRV Follow Speed"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_speed.transAxes, frameon=True, framealpha=0.9
            )
            ax_speed.set_ylabel("Speed (m/s)", fontsize=plot_params['ylabel_fontsize'])
            ax_speed.set_xlim(0, max_time)
            ax_speed.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_speed.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 >= 40:
                ax_speed.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
       
            # Plot spacing
            ax_spacing = axes_new[plot_idx * 2 + 1]
            ax_spacing.plot(time_shifted, spacing, label="Actual Spacing", color=new_dataset_color_map[condition], linestyle='-', linewidth=1)
            ax_spacing.plot(ovrv_time, ovrv_spacing, label="OVRV Spacing", color='black', linestyle=':', linewidth=1)
            ax_spacing.set_title(
                f"{dataset_name}_{condition}_{os.path.basename(csv_file)} Spacing\nSpacing RMSE: {spacing_rmse:.2f} m",
                fontsize=plot_params['title_fontsize'], wrap=True
            )
            ax_spacing.legend(
                ["Actual Spacing", "OVRV Spacing"],
                fontsize=plot_params['legend_fontsize'], loc='upper left',
                bbox_to_anchor=(0.0, 1.0), bbox_transform=ax_spacing.transAxes, frameon=True, framealpha=0.9
            )
            ax_spacing.set_ylabel("Spacing (m)", fontsize=plot_params['ylabel_fontsize'])
            ax_spacing.set_xlim(0, max_time)
            ax_spacing.grid(True, linestyle=plot_params['grid_linestyle'], alpha=plot_params['grid_alpha'])
            ax_spacing.tick_params(axis='both', labelsize=plot_params['tick_labelsize'])
            if plot_idx * 2 + 1 >= 40:
                ax_spacing.set_xlabel("Time (s)", fontsize=plot_params['xlabel_fontsize'])
       
            plot_idx += 1
        except Exception as e:
            print(f"Error processing {csv_file}: {e}")

# Adjust new datasets layout
fig_new.suptitle("New Datasets Speed and Spacing Trajectories with Optimized OVRV Parameters", fontsize=14)
fig_new.tight_layout(rect=[0, 0, 1, 0.95])
fig_new.savefig(os.path.join(output_folder, "new_datasets_trajectories_optimized.png"), dpi=300, bbox_inches='tight')
plt.close(fig_new)

print(f"\nPlots saved to: {output_folder}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.382085633884409
            Iterations: 13
            Function evaluations: 72
            Gradient evaluations: 13


C:\Users\yliu117\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\yliu117\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.716149903863034
            Iterations: 14
            Function evaluations: 80
            Gradient evaluations: 14
Optimization terminated successfully    (Exit mode 0)
            Current function value: 12.813412748573867
            Iterations: 15
            Function evaluations: 80
            Gradient evaluations: 15


C:\Users\yliu117\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 17.422084361680945
            Iterations: 14
            Function evaluations: 77
            Gradient evaluations: 14
Optimization terminated successfully    (Exit mode 0)
            Current function value: 19.335545276708466
            Iterations: 14
            Function evaluations: 78
            Gradient evaluations: 13


...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.498643791061458
            Iterations: 24
            Function evaluations: 128
            Gradient evaluations: 24


C:\Users\yliu117\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 27.20259590999545
            Iterations: 21
            Function evaluations: 119
            Gradient evaluations: 21


C:\Users\yliu117\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 63.537205095827524
            Iterations: 26
            Function evaluations: 146
            Gradient evaluations: 25

Best OVRV Parameters and RMSEs for ICE Vehicles:

Vehicle: A, Setting: Min
k1: 0.0601 1/s²
k2: 0.2164 1/s
eta: 10.00 m
tau: 0.97 s
RMSEs:
 Condition dip_min:
 Speed RMSE: 0.31 m/s
 Spacing RMSE: 1.57 m
 Condition osc_min:
 Speed RMSE: 0.18 m/s
 Spacing RMSE: 0.83 m
 Condition stepH_min:
 Speed RMSE: 0.10 m/s
 Spacing RMSE: 0.99 m
 Condition stepL_min:
 Speed RMSE: 0.14 m/s
 Spacing RMSE: 0.99 m

Vehicle: A, Setting: Max
k1: 0.0185 1/s²
k2: 0.1465 1/s
eta: 10.00 m
tau: 2.09 s
RMSEs:
 Condition dip_max:
 Speed RMSE: 0.36 m/s
 Spacing RMSE: 2.30 m
 Condition osc_max:
 Speed RMSE: 0.29 m/s
 Spacing RMSE: 2.17 m
 Condition stepH_max:
 Speed RMSE: 0.12 m/s
 Spacing RMSE: 1.04 m
 Condition stepL_max:
 Speed RMSE: 0.22 m/s
 Spacing RMSE: 2.20 m

Vehicle: B, Setting: Min
k1: 0.0468 1/s²